In [2]:
################################  Storage_Optimization.ipynb  ####################################
# Author: Sukhendu Sain
# Description: Analyse the Data
# Data: 23-Nov-2024
#################################################################################

In [3]:
# Import Necessary Libraries, Utils, and Config Files
import utils
from config import *
import pandas as pd
import numpy as np
import os, re
import matplotlib.pyplot as plt
# import importlib
# importlib.reload(utils)

# Data Import and Clean

In [ ]:
#### Read FILE:: (AKINS FoMoCo_Piece_Sales_112222_YTD.xlsx) into Dataframe
df_Akins = utils.read_excel(AKINS_FOMO_FILE_PATH)
df_Akins['Original Part#'] = df_Akins['Part#']
df_Akins['Part#'] = df_Akins['Part#'].apply(lambda a: "".join(str(a).split('-')))
if print_df_after_import: utils.print_df(df_Akins, 200) # Print the Dataframe
# ~1-2secs

In [ ]:
#### Read FILE:: (GPARTS Part Measures.xlsx) into Dataframe
df_Gparts = utils.read_excel(GPARTS_FILE_PATH)
if print_df_after_import: utils.print_df(df_Gparts) # Print the Dataframe
# ~50-60secs

In [ ]:
#### Read FILE:: (Wholesale JAN_Oct_Parts_Ranking_Counter_Invoices_All_Brands.xlsx) into Dataframe
df_Wholesale = utils.read_excel(WHOLESALE_FILE_PATH)

# Clean the Wholesale Dataframe
df_Wholesale['Description'] = df_Wholesale['Description'].astype(str)
df_Wholesale = df_Wholesale.drop(columns=[col for col in df_Wholesale.columns if 'Unnamed' in col], inplace=False)
df_Wholesale = df_Wholesale[(df_Wholesale['Vendor'] == 'FOR') | (df_Wholesale['Vendor'] == 'CHR')].reset_index()
df_Wholesale.loc[df_Wholesale['Description'].apply(lambda x: len(x.split("      ")) > 1), 'Avg. Cost'] = df_Wholesale['Description'].apply(lambda x: [i for i in x.strip().split("      ")][-1])
df_Wholesale.loc[df_Wholesale['Description'].apply(lambda x: len(x.split("      ")) > 1), 'Description'] = df_Wholesale['Description'].apply(lambda x: "     ".join([i for i in x.strip().split("      ")][:-1]))

if print_df_after_import: utils.print_df(df_Wholesale, 14900) # Print the Dataframe
# ~12-15secs

In [ ]:
#### Read FILE:: (Service JAN_Oct_Parts_Ranking_ROs_All_Brands.xlsx) into Dataframe
df_Service = utils.read_excel(SERVICE_FILE_PATH)

# Clean the Service Dataframe
df_Service['Description'] = df_Service['Description'].astype(str)
df_Service = df_Service.drop(columns=[col for col in df_Service.columns if 'Unnamed' in col], inplace=False)
df_Service = df_Service[(df_Service['Vendor'] == 'FOR') | (df_Service['Vendor'] == 'CHR')].reset_index()
df_Service.loc[df_Service['Description'].apply(lambda x: len(x.split("      ")) > 1), 'Avg. Cost'] = df_Service['Description'].apply(lambda x: [i for i in x.strip().split("      ")][-1])
df_Service.loc[df_Service['Description'].apply(lambda x: len(x.split("      ")) > 1), 'Description'] = df_Service['Description'].apply(lambda x: "     ".join([i for i in x.strip().split("      ")][:-1]))
#df_Service.loc[df_Service['Qty Sold'].apply(lambda x: len(str(x).split("      ")) > 1), 'Dollars Sold'] = df_Service['Qty Sold'].apply(lambda x: [i for i in str(x).strip().split("      ")][-1])
#df_Service.loc[df_Service['Qty Sold'].apply(lambda x: len(str(x).split("      ")) > 1), 'Qty Sold'] = df_Service['Qty Sold'].apply(lambda x: "     ".join([i for i in str(x).strip().split("      ")][:-1]))

if print_df_after_import: utils.print_df(df_Service, 100) # Print the Dataframe
# ~5-6secs

In [9]:
#### Read FILE:: (Counter Pad) into Dataframe

# Data Analysis


### Akins File Analysis

In [ ]:
## Sort 'Sold Pcs' Column Descending
df_Akins = df_Akins.sort_values('Sold Pcs ', ascending=False)
if print_df_data_analyse: utils.print_df(df_Akins, 100) # Print the Dataframe

In [ ]:
## Sum up the 'Sold Pcs' for each unique 'Part Desc.'
part_type_sold_sum = df_Akins.groupby('Description')['Sold Pcs '].sum().reset_index()
part_type_sold_sum.columns = ['Part Desc.', 'Total Sold Pcs.']

# Sort the part_type_sold_sum by Total Sold Pcs. in descending order
part_type_sold_sum = part_type_sold_sum.sort_values('Total Sold Pcs.', ascending=False)

if print_df_data_analyse: utils.print_df(part_type_sold_sum, 5)

In [ ]:
## Visualize above part_type_sold_sum using Bar, One Fig of Top 10 Most Sold Part Types and One for Top 50
top_10 = part_type_sold_sum.head(10)
top_50 = part_type_sold_sum.head(50)

# Create the bar chart for Top 10
plt.figure(figsize=(8, 6))  # Set the figure size
plt.bar(top_10['Part Desc.'], top_10['Total Sold Pcs.'], width=0.5)

# Customize the chart
plt.title('Top 10 Part Types by Total Sold Pieces (Akins)', fontsize=20)
plt.xlabel('Part Description', fontsize=16)
plt.ylabel('Total Sold Pieces', fontsize=16)
plt.xticks(rotation=90)

# Add value labels on top of each bar
for i, v in enumerate(top_10['Total Sold Pcs.']):
    plt.text(i, v, str(v), ha='center', va='bottom')

plt.tight_layout()
plt.show()


# Create the bar chart for Top 50
plt.figure(figsize=(9, 6))  # Set the figure size
plt.bar(top_50['Part Desc.'], top_50['Total Sold Pcs.'], width=0.5)

# Customize the chart
plt.title('Top 50 Part Types by Total Sold Pieces (Akins)', fontsize=20)
plt.xlabel('Part Description', fontsize=16)
plt.ylabel('Total Sold Pieces', fontsize=16)
plt.xticks(rotation=90)
plt.grid(axis='x', linestyle='--', alpha=1)
plt.grid(axis='y', linestyle='--', alpha=1)

plt.tight_layout()
plt.show()

### GParts File Analysis

In [ ]:
## Analyze 'Active' Column of GParts
total_rows = df_Gparts.shape[0]
active = df_Gparts[df_Gparts["Is Active?"] == 'Yes'].shape[0]
active_percent = ( df_Gparts[df_Gparts["Is Active?"] == 'Yes'].shape[0]/total_rows ) * 100

# Print the Counts/Percentage
print(f"Active Parts: {active}; Active Percentage: {active_percent}%")
print(f"Not Active Parts: {total_rows - active}; Not Active Percentage: {100 - active_percent}%")

# Visualize
labels = ['Active', 'Not Active'] # Create labels and sizes for the pie chart
sizes = [active_percent, 100 - active_percent]

plt.figure(figsize=(10, 8)) # Create the pie chart
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, textprops={'fontsize': 23})

plt.show() # Show the chart

In [ ]:
## Find Rows with 0 in either Dimensions
# Depth (Column Here) = Length (In Docs)

# Count no. of Rows with 0 in either Dimensions; no_all share count with other 3
no_depth = df_Gparts[df_Gparts["Prod Att - Length"] == 0].shape[0]
no_width = df_Gparts[df_Gparts["Prod Att- Width"] == 0].shape[0]
no_height = df_Gparts[df_Gparts["Prod Att - Height"] == 0].shape[0]
no_all = df_Gparts[(df_Gparts["Prod Att - Height"] == 0) & (df_Gparts["Prod Att- Width"] == 0) & (df_Gparts["Prod Att - Length"] == 0)].shape[0]

# Calculate percentages
total_rows = df_Gparts.shape[0]
percent_no_depth = (no_depth / total_rows) * 100
percent_no_width = (no_width / total_rows) * 100
percent_no_height = (no_height / total_rows) * 100
percent_no_all = (no_all / total_rows) * 100

# Count 'Active' Parts having No Dimensions and Dimensions
No_Dim_Active = df_Gparts[(df_Gparts["Prod Att - Height"] == 0) & (df_Gparts["Is Active?"] == 'Yes')].shape[0]
No_Dim_Not_Active = df_Gparts[(df_Gparts["Prod Att - Height"] == 0) & (df_Gparts["Is Active?"] == 'No')].shape[0]
No_Dim_Active_Percent = (No_Dim_Active/no_all) * 100
No_Dim_Not_Active_Percent = (No_Dim_Not_Active/no_all) * 100
With_Dim_Active = df_Gparts[(df_Gparts["Prod Att - Height"] != 0) & (df_Gparts["Is Active?"] == 'Yes')].shape[0]
With_Dim_Not_Active = df_Gparts[(df_Gparts["Prod Att - Height"] != 0) & (df_Gparts["Is Active?"] == 'No')].shape[0]
With_Dim_Active_Percent = (With_Dim_Active/(total_rows - no_all)) * 100
With_Dim_Not_Active_Percent = (With_Dim_Not_Active/(total_rows - no_all)) * 100

# Print the Counts/Percentages
print(f"No Length/Depth: {round(no_depth/total_rows*100, 2)}%; No Width: {round(no_width/total_rows*100, 2)}%; No Height: {round(no_height/total_rows*100, 2)}%; No Dimensions: {round(percent_no_all, 2)}%;")
print(f"Active Parts of No Dimensions: {No_Dim_Active}; Percentage with respect to Parts without Dims: {round(No_Dim_Active_Percent, 2)}%")
print(f"Not Active Parts of No Dimensions: {No_Dim_Not_Active}; Percentage with respect to Parts without Dims: {round(No_Dim_Not_Active_Percent, 2)}%")
print(f"Active Parts with Dimensions: {With_Dim_Active}; Percentage with respect to Parts with Dims: {round(With_Dim_Active_Percent, 2)}%")
print(f"Not Active Parts with Dimensions: {With_Dim_Not_Active}; Percentage with respect to Parts with Dims: {round(With_Dim_Not_Active_Percent, 2)}%")

# Here we find out that a row, if containing 0 in 1 dimension, has 0 in all, or
# A row has either all or none dimensions
# 16% of Rows has 0 Dimensions

In [ ]:
# Visualize
labels = ['Active', 'Not Active'] # Create labels and sizes for the pie chart
sizes = [No_Dim_Active_Percent, No_Dim_Not_Active_Percent]

plt.figure(figsize=(10, 8)) # Create the pie chart
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, textprops={'fontsize': 20})

plt.show() # Show the chart

In [ ]:
## Visualize above No Dimensions Data

# Create the bar graph
plt.figure(figsize=(10, 6)) 
bars = ['Total Rows', 'No Length/Depth', 'No Width', 'No Height']
heights = [100, percent_no_depth, percent_no_width, percent_no_height]

plt.bar(bars, heights)
plt.title(f'Distribution of Rows with No Dimensions', fontsize=16)
plt.ylabel('Percentage of Rows')
plt.xticks(rotation=45, fontsize=16)

# Add labels to each bar
for i, v in enumerate(heights):
    plt.text(i, v, str(round(v, 2)) + '%', ha='center', va='bottom')

# Show the legend and display the chart
plt.tight_layout()
plt.show()

In [ ]:
## Visualize everything Analyzed above in a single Stacked Bar Chart

categories = ['Total Rows', 'Dimensional Rows', 'No Dimensional Rows']
values1 = [active, With_Dim_Active, No_Dim_Active]
values2 = [total_rows - active, With_Dim_Not_Active, No_Dim_Not_Active]

# Create figure and axis objects
fig, ax = plt.subplots(figsize=(8, 6))

# Plot the stacked bar chart
ax.bar(categories, values1, label='Active', width = 0.3)
ax.bar(categories, values2, bottom=values1, label='Not Active', width = 0.3)

for i, v in enumerate([total_rows, With_Dim_Active+With_Dim_Not_Active, No_Dim_Active+No_Dim_Not_Active]):
    plt.text(i, v, f'Total: {str(v)}', ha='center', va='bottom')

# Customize the plot
ax.set_ylabel('Values')
ax.set_title('Bar Chart of Active and Non-Active Parts (GParts)')
ax.legend()

ax.set_yticks(np.linspace(0, 231045, 20))  # Set more ticks on y-axis

plt.grid(axis='x', linestyle='--', alpha=1)
plt.grid(axis='y', linestyle='--', alpha=1)

# Display the chart
plt.tight_layout()
plt.show()

### Wholesale Files Analysis

In [ ]:
## Sort 'Sold' Column Descending
df_Wholesale = df_Wholesale.sort_values('Sold', ascending=False)
if print_df_data_analyse: utils.print_df(df_Wholesale, 100) # Print the Dataframe

In [ ]:
## Count the number of negative sold pcs.
neg_sold_count = df_Wholesale[df_Wholesale['Sold'] < 0].shape[0]
print(f"Number of Negative Sold Values: {neg_sold_count}")

In [ ]:
## Sum up the 'Gross Profit' for each unique 'Part Desc.'
part_type_profit_sum = df_Wholesale.groupby('Description')['Gross Profit'].sum().reset_index()
part_type_profit_sum.columns = ['Part Desc.', 'Total Gross Profit']
part_type_profit_sum['Total Gross Profit'] = pd.to_numeric(part_type_profit_sum['Total Gross Profit'], errors='coerce').round()

# Sort the part_type_sold_sum by Total Sold Pcs. in descending order
part_type_profit_sum = part_type_profit_sum.sort_values('Total Gross Profit', ascending=False)
part_type_profit_sum['Total Gross Profit'] = part_type_profit_sum['Total Gross Profit'].round(-1)

if print_df_data_analyse: utils.print_df(part_type_profit_sum, 5)

# Visualize using Bar of Top 10
top_10 = part_type_profit_sum.head(10)

# Create the bar chart for Top 10
plt.figure(figsize=(8, 6))  # Set the figure size
plt.bar(top_10['Part Desc.'], top_10['Total Gross Profit'], width=0.5)

# Customize the chart
plt.title('Top 10 Part Types by Total Gross Profit (Wholesale)', fontsize=20)
plt.xlabel('Part Description', fontsize=16)
plt.ylabel('Total Gross Profit', fontsize=16)
plt.xticks(rotation=90)

# Add value labels on top of each bar
for i, v in enumerate(top_10['Total Gross Profit']):
    plt.text(i, v, str(v), ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
## Sum up the 'Sold' for each unique 'Part Desc.'
part_type_sold_sum = df_Wholesale.groupby('Description')['Sold'].sum().reset_index()
part_type_sold_sum.columns = ['Part Desc.', 'Total Sold Pcs.']

# Sort the part_type_sold_sum by Total Sold Pcs. in descending order
part_type_sold_sum = part_type_sold_sum.sort_values('Total Sold Pcs.', ascending=False)

if print_df_data_analyse: utils.print_df(part_type_sold_sum, 5)

In [ ]:
## Visualize above part_type_sold_sum using Bar, One Fig of Top 10 Most Sold Part Types and One for Top 50
top_10 = part_type_sold_sum.head(10)
top_50 = part_type_sold_sum.head(50)

# Create the bar chart for Top 10
plt.figure(figsize=(8, 6))  # Set the figure size
plt.bar(top_10['Part Desc.'], top_10['Total Sold Pcs.'], width=0.5)

# Customize the chart
plt.title('Top 10 Part Types by Total Sold Pieces (Wholesale)', fontsize=20)
plt.xlabel('Part Description', fontsize=16)
plt.ylabel('Total Sold Pieces', fontsize=16)
plt.xticks(rotation=90)

# Add value labels on top of each bar
for i, v in enumerate(top_10['Total Sold Pcs.']):
    plt.text(i, v, str(v), ha='center', va='bottom')

plt.tight_layout()
plt.show()


# Create the bar chart for Top 50
plt.figure(figsize=(9, 6))  # Set the figure size
plt.bar(top_50['Part Desc.'], top_50['Total Sold Pcs.'], width=0.5)

# Customize the chart
plt.title('Top 50 Part Types by Total Sold Pieces (Wholesale)', fontsize=20)
plt.xlabel('Part Description', fontsize=16)
plt.ylabel('Total Sold Pieces', fontsize=16)
plt.xticks(rotation=90)
plt.grid(axis='x', linestyle='--', alpha=1)
plt.grid(axis='y', linestyle='--', alpha=1)

plt.tight_layout()
plt.show()

### Service File Analysis

In [ ]:
## Count the number of negative sold pcs.
neg_sold_count = df_Service[df_Service['Qty Sold'] < 0].shape[0]
print(f"Number of Negative Sold Values: {neg_sold_count}")

In [ ]:
## Sum up the 'Gross Profit' for each unique 'Part Desc.'
part_type_profit_sum = df_Service.groupby('Description')['Gross Profit'].sum().reset_index()
part_type_profit_sum.columns = ['Part Desc.', 'Total Gross Profit']
part_type_profit_sum['Total Gross Profit'] = pd.to_numeric(part_type_profit_sum['Total Gross Profit'], errors='coerce').round()

# Sort the part_type_sold_sum by Total Sold Pcs. in descending order
part_type_profit_sum = part_type_profit_sum.sort_values('Total Gross Profit', ascending=False)
part_type_profit_sum['Total Gross Profit'] = part_type_profit_sum['Total Gross Profit'].round(-1)

if print_df_data_analyse: utils.print_df(part_type_profit_sum, 5)

# Visualize using Bar of Top 10
top_10 = part_type_profit_sum.head(10)

# Create the bar chart for Top 10
plt.figure(figsize=(8, 6))  # Set the figure size
plt.bar(top_10['Part Desc.'], top_10['Total Gross Profit'], width=0.5)

# Customize the chart
plt.title('Top 10 Part Types by Total Gross Profit (Service)', fontsize=20)
plt.xlabel('Part Description', fontsize=16)
plt.ylabel('Total Gross Profit', fontsize=16)
plt.xticks(rotation=90)

# Add value labels on top of each bar
for i, v in enumerate(top_10['Total Gross Profit']):
    plt.text(i, v, str(v), ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
## Sum up the 'Sold' for each unique 'Part Desc.'
part_type_sold_sum = df_Service.groupby('Description')['Qty Sold'].sum().reset_index()
part_type_sold_sum.columns = ['Part Desc.', 'Total Sold Pcs.']

# Sort the part_type_sold_sum by Total Sold Pcs. in descending order
part_type_sold_sum = part_type_sold_sum.sort_values('Total Sold Pcs.', ascending=False)

if print_df_data_analyse: utils.print_df(part_type_sold_sum, 5)

In [ ]:
## Visualize above part_type_sold_sum using Bar, One Fig of Top 10 Most Sold Part Types and One for Top 50
top_10 = part_type_sold_sum.head(10)
top_50 = part_type_sold_sum.head(50)

# Create the bar chart for Top 10
plt.figure(figsize=(8, 6))  # Set the figure size
plt.bar(top_10['Part Desc.'], top_10['Total Sold Pcs.'], width=0.5)

# Customize the chart
plt.title('Top 10 Part Types by Total Sold Pieces (Service)', fontsize=20)
plt.xlabel('Part Description', fontsize=16)
plt.ylabel('Total Sold Pieces', fontsize=16)
plt.xticks(rotation=90)

# Add value labels on top of each bar
for i, v in enumerate(top_10['Total Sold Pcs.']):
    plt.text(i, v, str(v), ha='center', va='bottom')

plt.tight_layout()
plt.show()


# Create the bar chart for Top 50
plt.figure(figsize=(9, 6))  # Set the figure size
plt.bar(top_50['Part Desc.'], top_50['Total Sold Pcs.'], width=0.5)

# Customize the chart
plt.title('Top 50 Part Types by Total Sold Pieces (Service)', fontsize=20)
plt.xlabel('Part Description', fontsize=16)
plt.ylabel('Total Sold Pieces', fontsize=16)
plt.xticks(rotation=90)
plt.grid(axis='x', linestyle='--', alpha=1)
plt.grid(axis='y', linestyle='--', alpha=1)

plt.tight_layout()
plt.show()

### Find Matching Part Numbers between Gparts and each of other Files

In [ ]:
## Find Number of Matching Part Numbers between GParts and Akins

dfs_to_match = [df_Akins, df_Gparts]

# Part# Column Name of DFs to Match
part_number_columns = ['Part#', 'Svc Part Number']

# Find common part numbers
common_part_numbers = set(dfs_to_match[0][part_number_columns[0]])
for i in range(1, len(dfs_to_match)):
    common_part_numbers &= set(dfs_to_match[i][part_number_columns[i]])

print(f"Num of Part numbers common to all DataFrames: {len(common_part_numbers)}")
print(f"Matching Part Numbers: {common_part_numbers}")
# 10283 Matching

In [ ]:
## Find Number of Matching Part Numbers between GParts and Wholesale

dfs_to_match = [df_Wholesale, df_Gparts]

# Part# Column Name of DFs to Match
part_number_columns = ['Part Number', 'Svc Part Number']

# Find common part numbers
common_part_numbers = set(dfs_to_match[0][part_number_columns[0]])
for i in range(1, len(dfs_to_match)):
    common_part_numbers &= set(dfs_to_match[i][part_number_columns[i]])

print(f"Num of Part numbers common to all DataFrames: {len(common_part_numbers)}")
print(f"Matching Part Numbers: {common_part_numbers}")
# 20763 Matching

In [ ]:
## Find Number of Matching Part Numbers between GParts and Service

dfs_to_match = [df_Service, df_Gparts]

# Part# Column Name of DFs to Match
part_number_columns = ['* indicates a superseded part\nPart Number', 'Svc Part Number']

# Find common part numbers
common_part_numbers = set(dfs_to_match[0][part_number_columns[0]])
for i in range(1, len(dfs_to_match)):
    common_part_numbers &= set(dfs_to_match[i][part_number_columns[i]])

print(f"Num of Part numbers common to all DataFrames: {len(common_part_numbers)}")
print(f"Matching Part Numbers: {common_part_numbers}")
# 12926 Matching

In [ ]:
## Find Number of Matching Part Numbers between Akins and Wholesale

dfs_to_match = [df_Akins, df_Wholesale]

# Part# Column Name of DFs to Match
part_number_columns = ['Part#', 'Part Number']

# Find common part numbers
common_part_numbers = set(dfs_to_match[0][part_number_columns[0]])
for i in range(1, len(dfs_to_match)):
    common_part_numbers &= set(dfs_to_match[i][part_number_columns[i]])

print(f"Num of Part numbers common to all DataFrames: {len(common_part_numbers)}")
print(f"Matching Part Numbers: {common_part_numbers}")
# 6884 Matching

In [ ]:
## Find Number of Matching Part Numbers between Service and Wholesale

dfs_to_match = [df_Service, df_Wholesale]

# Part# Column Name of DFs to Match
part_number_columns = ['* indicates a superseded part\nPart Number', 'Part Number']

# Find common part numbers
common_part_numbers = set(dfs_to_match[0][part_number_columns[0]])
for i in range(1, len(dfs_to_match)):
    common_part_numbers &= set(dfs_to_match[i][part_number_columns[i]])

print(f"Num of Part numbers common to all DataFrames: {len(common_part_numbers)}")
print(f"Matching Part Numbers: {common_part_numbers}")
# 6884 Matching

In [ ]:
## Check Sold between Wholesale and Service Matching Part Numbers

test = []
for cpt in common_part_numbers:
    test.append([cpt, df_Wholesale[df_Wholesale['Part Number'] == cpt]['Sold'].values[0], df_Service[df_Service['* indicates a superseded part\nPart Number'] == cpt]['Qty Sold'].values[0]])

df_test = pd.DataFrame(test)
utils.print_df(df_test)

In [ ]:
## Check Description between Wholesale and Service Matching Part Numbers

test = []
for cpt in common_part_numbers:
    test.append([cpt, df_Wholesale[df_Wholesale['Part Number'] == cpt]['Description'].values[0], df_Service[df_Service['* indicates a superseded part\nPart Number'] == cpt]['Description'].values[0]])

df_test = pd.DataFrame(test)
utils.print_df(df_test)

In [ ]:
df_test[df_test[1] != df_test[2]]

In [ ]:
# It will have the Columns - 'Part Number', 'Part Desc.', 'Active', 'Sold (Pcs.)', '0Dimensions', 'Length/Depth', 'Width', 'Height', 'Zone', 'Storage Type', 'Sub Storage', 'Number of Storage needed'
# It will have all the rows with common part nos. from all 4 Files, having Appropriate Sold Pcs. Values, and Dimensions

main_list = []

gParts_PartNos = set(df_Gparts['Svc Part Number'])

# common_part_numbers = gParts_PartNos & set(df_Akins['Part#'])
# for pn, pddesc, ac, s, ld, w, h in zip(common_part_numbers, df_Gparts["Svc Part Number Description"], df_Gparts['Is Active?'], df_Akins['Sold Pcs '], df_Gparts['Prod Att - Length'], df_Gparts['Prod Att- Width'], df_Gparts['Prod Att - Height']):
#     main_list.append([pn, pddesc, "Akins", ac, s, False, ld, w, h, "", "", "", ""])

common_part_numbers = gParts_PartNos & set(df_Wholesale['Part Number'])
for pn in common_part_numbers:
    part_row = df_Gparts[df_Gparts['Svc Part Number'] == pn]
    pddesc = part_row["Svc Part Number Description"]
    ac = part_row['Is Active?']
    s = df_Wholesale[df_Wholesale['Part Number'] == pn]['Sold']
    ld =  part_row['Prod Att - Length']
    w = part_row['Prod Att- Width']
    h = part_row['Prod Att - Height']
    main_list.append([pn, pddesc, "Wholesale", ac, s, False, ld, w, h, "", "", "", "", ""])


common_part_numbers = gParts_PartNos & set(df_Service['* indicates a superseded part\nPart Number'])
for pn in common_part_numbers:
    part_row = df_Gparts[df_Gparts['Svc Part Number'] == pn]
    pddesc = part_row["Svc Part Number Description"]
    ac = part_row['Is Active?']
    s = df_Service[df_Service['* indicates a superseded part\nPart Number'] == pn]['Sold Qty']
    ld =  part_row['Prod Att - Length']
    w = part_row['Prod Att- Width']
    h = part_row['Prod Att - Height']
    main_list.append([pn, pddesc, "Service", ac, s, False, ld, w, h, "", "", "", "", ""])

df_Main = pd.DataFrame(main_list)
df_Main.columns = ['Part#', 'Part Desc.', 'DataSource', 'Active', 'Sold', '0Dimensions', 'Depth', 'Width', 'Height', 'Zone', 'StorageType', 'SubStorage', 'Num. Storage Required', 'Bin Location']
#df_Main = df_Main.sort_values('Sold', ascending=False).reset_index()
df_Main["Zone"] = df_Main["Zone"].astype(str)
df_Main.loc[(df_Main["Depth"] == 0) | (df_Main["Height"] == 0) | (df_Main["Width"] == 0), "0Dimensions"] = True
#df_Main = df_Main[df_Main["0Dimensions"] == False].reset_index()
#df_Main.drop(['index', 'level_0'], axis=1, inplace=True)
#df_Main.drop(['index'], axis=1, inplace=True)


utils.print_df(df_Main)